In [20]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import glob
import math
import cv2
import os
import re

from tqdm import tqdm

In [21]:
INCLUDE_CHART_TYPE_IN_TEXT = False
ADD_EXTRA_DATA = True
IMG_DIR = "images"

In [22]:
df = pd.DataFrame(columns=["image_path", "target_text", "target_type", "source"])


In [23]:
coma_to_remove = []

target_text = []
chart_types = []
sources = []

x_axis_type = []
y_axis_type = []

target_text_x = []
target_text_y = []
image_paths = []

x_axis_types = []
y_axis_types = []

sus = []
cont = False
for annotation_path in tqdm(glob.glob("/home/pawel/Projects/benetech-making-graphs-accessible/data/train/annotations/*.json")):
    
    with open(annotation_path, "r") as annotation_file:
        annotation = json.load(annotation_file)
        img_path = annotation_path.replace(".json", ".jpg").replace("annotations", IMG_DIR)
    if any([id_ in annotation_path for id_ in ["944014b8ce8c", "ff721b3ae2eb", "89d24be7fcb0", "b56e10126c31", "e2ee063cb374", "4e2da02bbe09", "5918579d6abf", "c9eccc00f7da", "4c61a11da10a", "5d4dd273a476"]]):
        continue    
    
    
    x_axis_type = annotation["axes"]["x-axis"]["values-type"]
    y_axis_type = annotation["axes"]["y-axis"]["values-type"]
        
    x_data_series = [xy["x"] for xy in annotation["data-series"]]
    y_data_series = [xy["y"] for xy in annotation["data-series"]]
    
    if y_axis_type == "numerical":
        if any(math.isnan(y) for y in y_data_series): 
            continue
    
    x_ticks_ids = [tick["id"] for tick in annotation["axes"]["x-axis"]["ticks"]]
    y_ticks_ids = [tick["id"] for tick in annotation["axes"]["y-axis"]["ticks"]]
    
    x_axis_ticks_dict = {element["id"]: element["text"] for element in annotation["text"] if element["id"] in x_ticks_ids}
    y_axis_ticks_dict = {element["id"]: element["text"] for element in annotation["text"] if element["id"] in y_ticks_ids}
    
    x_axis_ticks = [x_axis_ticks_dict[tick_id] for tick_id in x_ticks_ids]
    y_axis_ticks = [y_axis_ticks_dict[tick_id] for tick_id in y_ticks_ids]
    
    
    if annotation["chart-type"] == "horizontal_bar" or "9c6d595e51d6" in annotation_path:
        x_axis_ticks, y_axis_ticks = y_axis_ticks, x_axis_ticks
        
    if any([id_ in annotation_path for id_ in ["496de625e57a", "6ce4bc728dd5", "d0cf883b1e13", "aa9df520a5f2", "04296b42ba61", "733b9b19e09a", "f3b0463765b2", "db54e3ef505f"]]):
        x_data_series = [round(x, 1) for x in x_data_series if x not in x_axis_ticks]
        y_data_series = [round(y, 1) for y in y_data_series if y not in y_axis_ticks]
        continue
    
    if any([id_ in annotation_path for id_ in ["6a92d147f4d5", "3968efe9cbfc", "e6ccc2703066", "fa91f20f277d"]]):
        x_data_series = [round(x, 1) for x in x_data_series if x not in x_axis_ticks]
        y_data_series = [int(float(y)) for y in y_data_series if y not in y_axis_ticks]
        continue
    
    if x_axis_type == "numerical":
        x_axis_ticks = [re.sub("[ %$]", "", tick) for tick in x_axis_ticks]
        x_axis_ticks = [re.sub(",", "", tick) if re.search(",\d{3}", tick) else tick for tick in x_axis_ticks]
                
        decimal_points_count = max([len(re.search(r"(\.)(\d+)", tick).group(2)) for tick in x_axis_ticks if re.search(r"(\.)(\d+)", tick) is not None] + [0])
        x_axis_ticks = [float(tick) if re.findall(r"[\.,]", tick) else int(tick) for tick in x_axis_ticks]
        
        x_diff = abs(x_axis_ticks[0] - x_axis_ticks[1])
        
        x_min = min(x_axis_ticks)
        x_max = max(x_axis_ticks)
        
        x_data_series = [round(x, decimal_points_count + 1) if (x_diff < 10 and decimal_points_count > 0) else int(round(float(x))) for x in x_data_series]
        
        x_series_min = min(x_data_series)
        x_series_max = max(x_data_series)
        
        if x_series_min < x_min or x_series_max > x_max:
            continue

        
    if y_axis_type == "numerical":
        y_axis_ticks = [str(eval(tick)) if re.search(r"[Ee][\+\-]", tick) else tick for tick in y_axis_ticks]
        y_axis_ticks = [re.sub(r"(\d)\.(\d{3}),(00)", r"\1\2\.\3", tick) for tick in y_axis_ticks]
        y_axis_ticks = [re.sub("[ %$]", "", tick) for tick in y_axis_ticks]
        y_axis_ticks = [tick.replace("..", ".") for tick in y_axis_ticks]
        y_axis_ticks = [tick.replace(",", ".") for tick in y_axis_ticks]
        
        one_label_has_dot = sum(["." in tick for tick in y_axis_ticks]) == 1
        
        y_axis_ticks = [tick.replace("5.00.000", "500.000").replace("1.500.00", "1500000").replace("1.00.000", "1000000") for tick in y_axis_ticks]
        
        if one_label_has_dot and len(y_axis_ticks) > 3 and not any([re.search(r"0[\.,]\d", tick) for tick in y_axis_ticks]):
            y_axis_ticks = [tick.replace(".", "") for tick in y_axis_ticks]
            
        y_axis_ticks = [tick.replace(".", "") if re.search(r"\d{3}\.\d{3}$", tick) else tick for tick in y_axis_ticks]
            
        if any([id_ in annotation_path for id_ in ["7a46ec28cc2c", "d75a8f2a99db", "d767b5908804"]]):
            y_axis_ticks = [tick[:-1] if re.search(r"\d\.$", tick) else tick for tick in y_axis_ticks]
        
        if any([id_ in annotation_path for id_ in ["0278b786a87e"]]):
            y_axis_ticks = [tick[1:] if re.search(r"^\.\d", tick) else tick for tick in y_axis_ticks]
        
        if "eecf40ff67c5" in annotation_path:
            y_axis_ticks = [tick.replace('2.5000.0', '25000.0') for tick in y_axis_ticks]
        
        if any([id_ in annotation_path for id_ in ["a3550cf6248e"]]):
            y_axis_ticks = [y.replace("..", "") for y in y_axis_ticks]
            
        decimal_points_count = max([len(re.search(r"(\.)(\d+)", tick).group(2)) for tick in y_axis_ticks if re.search(r"(\.)(\d+)", tick) is not None] + [0])
        y_axis_ticks = [float(tick) if re.findall(r"[\.,]", tick) else int(tick) for tick in y_axis_ticks]
        
        y_diff = abs(y_axis_ticks[0] - y_axis_ticks[1])
        
        y_min = min(y_axis_ticks)
        y_max = max(y_axis_ticks)
        
        y_data_series = [round(y, decimal_points_count + 1) if (y_diff < 10 and decimal_points_count > 0) else int(float(y)) for y in y_data_series]

        y_series_min = min(y_data_series)
        y_series_max = max(y_data_series)
        
        if y_series_min < y_min or y_series_max > y_max:
            continue
    
    x_axis_types.append(x_axis_type)
    y_axis_types.append(y_axis_type)
    chart_types.append(annotation["chart-type"])
    sources.append(annotation["source"])
    image_paths.append(img_path)
    target_text_x.append(";".join(str(x) for x in x_data_series))
    target_text_y.append(";".join(str(y) for y in y_data_series))
    target_text.append((f'{annotation["chart-type"]} <0x0A> ' if INCLUDE_CHART_TYPE_IN_TEXT else '') + f"X | Y <0x0A> " + " <0x0A> ".join([f"{x} | {y}" for x, y in zip(x_data_series, y_data_series)]))

100%|██████████| 60578/60578 [00:09<00:00, 6244.32it/s]


In [24]:
df["image_path"] = image_paths

df["x_axis_type"] = x_axis_types
df["y_axis_type"] = y_axis_types

df["target_text_x"] = target_text_x
df["target_text_y"] = target_text_y

df["target_text"] = target_text
df["target_type"] = chart_types
df["source"] = sources

In [25]:
if ADD_EXTRA_DATA:
    if INCLUDE_CHART_TYPE_IN_TEXT:
        df = pd.concat([df, pd.read_csv("../data/extra_data_with_type.csv")], ignore_index=True)
    else:
        df = pd.concat([df, pd.read_csv("../data/extra_data.csv")], ignore_index=True)

In [26]:
df.to_csv(f"../data/df_{IMG_DIR}{'_with_type' if INCLUDE_CHART_TYPE_IN_TEXT else ''}{'_extra' if ADD_EXTRA_DATA else ''}.csv", index=False)